<a href="https://colab.research.google.com/github/Pooja-0708/GEN-AI/blob/main/gpt_2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate torchvision

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset

In [ ]:
# dataset used for fine tuning...
# https://huggingface.co/datasets/hakurei/open-instruct-v1
dataset = load_dataset("hakurei/open-instruct-v1", split='train')
print(dataset.to_pandas().sample(1))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

                                                   output  \
163256  One way to get rid of a headache quickly is by...   

                                     instruction input  
163256  How can I get rid of a headache quickly.        


In [ ]:
inst = "u r a calculator"
inpu = "6 + 4"
outp = "10"
prompt = f"{inst} {inpu} {outp}"
print(prompt)

u r a calculator 6 + 4 10


In [ ]:
def preprocess(row):
    row['prompt'] = f"{row['instruction']} {row['input']} {row['output']}"
    return row

In [ ]:
print(f"Before preprocessing: {dataset}")

Before preprocessing: Dataset({
    features: ['output', 'instruction', 'input'],
    num_rows: 498813
})


In [ ]:
dataset = dataset.map(preprocess, remove_columns=['instruction', 'input', 'output'])
print(f"After preprocessing: {dataset}")

Map:   0%|          | 0/498813 [00:00<?, ? examples/s]

After preprocessing: Dataset({
    features: ['prompt'],
    num_rows: 498813
})


In [ ]:
dataset = dataset.shuffle(seed=42).select(range(100000)).train_test_split(test_size=0.1)
print(f"After train test split: {dataset}")

After train test split: DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['prompt'],
        num_rows: 10000
    })
})


In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
# https://huggingface.co/microsoft/DialoGPT-medium
MODEL_NAME = 'microsoft/DialoGPT-medium'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
print(f"First row Before tokenizing: {train_dataset['prompt'][0]}")

First row Before tokenizing: Give two opposite opinions on the issue and support them with examples.  Issue: Should the government ban all guns?
Opposing Opinions:
- Yes, because it will reduce gun violence.


In [ ]:
def tokenize_dataset(dataset):
    tokenized_dataset = dataset.map(lambda row:tokenizer(row['prompt'], truncation=True, max_length=128), batched=True, remove_columns=['prompt'])
    return tokenized_dataset

In [ ]:
train_dataset = tokenize_dataset(train_dataset)
# test_dataset = tokenize_dataset(test_dataset)
print(f"tokenized train dataset: {train_dataset}")
print(f"First row After tokenizing: {train_dataset['input_ids'][0], train_dataset['attention_mask'][0]}")

Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

tokenized train dataset: Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 90000
})
First row After tokenizing: ([23318, 734, 6697, 9317, 319, 262, 2071, 290, 1104, 606, 351, 6096, 13, 220, 18232, 25, 10358, 262, 1230, 3958, 477, 6541, 30, 198, 27524, 2752, 8670, 259, 507, 25, 198, 12, 3363, 11, 780, 340, 481, 4646, 2485, 3685, 13], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir='./dialogpt2-instruct',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
print("training started...")
trainer.train()
print("training completed...")
trainer.save_model()
print("saved model...")

In [ ]:
# fine_tuned_model = AutoModelForCausalLM.from_pretrained('./dialogpt2-instruct').to('cuda')
fine_tuned_model = AutoModelForCausalLM.from_pretrained('TheFuzzyScientist/diabloGPT_open-instruct').to('cuda')

config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
def generate_text(prompt, model_selected):
    inputs = tokenizer.encode(prompt, return_tensors='pt').to('cuda')
    outputs = model_selected.generate(inputs, max_length=64, pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(outputs[0], skip_special_token=True, use_mps_device=True)
    return generated_text[: generated_text.rfind('.')+1]

In [ ]:
print("Generating text from base model... ")
print(generate_text("I am Arish", model))

Generating text from base model... 
I am Arish.


In [ ]:
print("Generating text from fine tuned model... ")
print(generate_text("I like to drink...", fine_tuned_model))

Generating text from fine tuned model... 
I like to drink...  I like to drink coffee.  I like to drink tea.  I like to drink coffee with milk.  I like to drink tea with milk.  I like to drink coffee with milk.  I like to drink tea with milk.  I like to drink coffee with milk.
